In [18]:
import os
from neo4j import GraphDatabase

In [2]:
# Replace with your Neo4j connection details
NEO4J_URI = os.getenv("NEO4J_URI", "bolt://localhost:7687")
NEO4J_USER = os.getenv("NEO4J_USER", "neo4j")
NEO4J_PASS = os.getenv("NEO4J_PASS", "")

In [9]:
driver = GraphDatabase.driver(NEO4J_URI, auth=("neo4j", "password"))

In [13]:
def get_person_details_from_neo4j(person_name: str):
    """
    Fetch details of a broker (Person node) from Neo4j.
    Returns all properties stored in the Person node.
    """
    query = """
    MATCH (p:Person)
    WHERE p.name = $name
    RETURN p
    """

    with driver.session() as session:
        result = session.run(query, name=person_name)
        record = result.single()
        if not record:
            return {"broker": None}

        person_node = record["p"]
        # convert Node properties to dict
        return {"broker": dict(person_node)}

In [15]:
get_person_details_from_neo4j("Mie Kim")

{'broker': {'role': "Seller's Representative",
  'name': 'Mie Kim',
  'title': 'Associate, Coldwell Banker',
  'url': '/people/mie-kim'}}

In [ ]:
"""Subject: Bridge Loan Opportunity
Hi Sabrina,
I wanted to share an off-market bridge opportunity that could be a great fit. The sponsor is experienced, the asset is cash flowing, and only a select group of lenders are reviewing this deal.
Shawna Menifee is seeking a $4.2M bridge loan to refinance existing debt on a property at 98–100 Bedford Avenue, Brooklyn, NY. The sponsor is targeting a 30-month loan term to complete renovations and full stabilization, with plans to refinance into long-term permanent debt once the repositioning is complete. He’s currently seeking pricing in the range of SOFR + 530 bps, depending on structure and recourse.
The building is 95% occupied, with residential rents slightly below market, offering near-term upside. Trader Joe’s anchors the retail portion under a long-term lease with annual escalations, providing steady cash flow. Planned upgrades include unit renovations and common-area improvements, which are expected to take approximately 24–28 months, after which the property is projected to achieve a stabilized value of around $38M.
Menifee brings 30+ years of NYC real estate experience and has developed and managed over $1.2B in multifamily and mixed-use assets.
 www.reynoldsproperties.com/about
Let me know a good time to connect to discuss this opportunity.
Best,
Bryan Son

"""

'Subject: Bridge Loan Opportunity\nHi Sabrina,\nI wanted to share an off-market bridge opportunity that could be a great fit. The sponsor is experienced, the asset is cash flowing, and only a select group of lenders are reviewing this deal.\nMie Kim is seeking a $4.2M bridge loan to refinance existing debt on a property at 98–100 Bedford Avenue, Brooklyn, NY. The sponsor is targeting a 30-month loan term to complete renovations and full stabilization, with plans to refinance into long-term permanent debt once the repositioning is complete. He’s currently seeking pricing in the range of SOFR + 530 bps, depending on structure and recourse.\nThe building is 95% occupied, with residential rents slightly below market, offering near-term upside. Trader Joe’s anchors the retail portion under a long-term lease with annual escalations, providing steady cash flow. Planned upgrades include unit renovations and common-area improvements, which are expected to take approximately 24–28 months, after 

In [ ]:
def fetch_broker_deals(broker_url: str):
    """
    Returns a list of deals a broker is involved in with all relevant info
    """
    query = """
    MATCH (p:Person {url: $broker_url})-[:PARTICIPATED_IN]->(d:Deal)
  OPTIONAL MATCH (d)-[:INVOLVES]->(prop:Property)
    OPTIONAL MATCH (p)-[:WORKS_FOR]->(org:Organization)
    RETURN d, 
           prop.address AS property_address, prop.url AS property_url,
           org.name AS organization_name, org.url AS organization_url, org.type as organization_type
    ORDER BY d.date DESC
    """
    with driver.session() as session:
        result = session.run(query, broker_url=broker_url)
        deals = []
        for record in result:
            deals.append({
                "deal_info": dict(record["d"]),
                "property_address": record["property_address"],
                "property_url": record["property_url"],
                "organization_name": record["organization_name"],
                "organization_url": record["organization_url"],
                "organization_type": record["organization_type"]
            })
    return (deals)

In [34]:
deals = fetch_broker_deals("/people/shawna-menifee")

{'d': <Node element_id='4:21c74eca-24d2-4b9b-a273-ff06f72301a3:13362' labels=frozenset({'Deal'}) properties={'date': '12/02/2021', 'floors': '1', 'square feet': '14,114', 'term years': '12', 'property': '477 Madison Ave', 'type': 'Lease', 'price per square foot': 'N/A', 'url': '/activity/477-madison-ave-lease-12022021-ariel-alternatives'}>, 'property_address': '477 Madison Ave, New York, NY, 10022', 'property_url': '/buildings/477-madison-ave-new-york-ny', 'organization_name': 'JLL', 'organization_url': '/organizations/jll', 'organization_type': 'Diversified'}
{'d': <Node element_id='4:21c74eca-24d2-4b9b-a273-ff06f72301a3:13362' labels=frozenset({'Deal'}) properties={'date': '12/02/2021', 'floors': '1', 'square feet': '14,114', 'term years': '12', 'property': '477 Madison Ave', 'type': 'Lease', 'price per square foot': 'N/A', 'url': '/activity/477-madison-ave-lease-12022021-ariel-alternatives'}>, 'property_address': '477 Madison Ave, New York, NY, 10022', 'property_url': '/buildings/47

In [35]:
deals

[{'deal_info': {'date': '12/02/2021',
   'floors': '1',
   'square feet': '14,114',
   'term years': '12',
   'property': '477 Madison Ave',
   'type': 'Lease',
   'price per square foot': 'N/A',
   'url': '/activity/477-madison-ave-lease-12022021-ariel-alternatives'},
  'property_address': '477 Madison Ave, New York, NY, 10022',
  'property_url': '/buildings/477-madison-ave-new-york-ny',
  'organization_name': 'JLL',
  'organization_url': '/organizations/jll',
  'organization_type': 'Diversified'},
 {'deal_info': {'date': '12/02/2021',
   'floors': '1',
   'square feet': '14,114',
   'term years': '12',
   'property': '477 Madison Ave',
   'type': 'Lease',
   'price per square foot': 'N/A',
   'url': '/activity/477-madison-ave-lease-12022021-ariel-alternatives'},
  'property_address': '477 Madison Ave, New York, NY, 10022',
  'property_url': '/buildings/477-madison-ave-new-york-ny',
  'organization_name': 'JLL',
  'organization_url': '/organizations/jll',
  'organization_type': 'Diver